### Neural Networks

In [2]:
from scratch.linear_algebra import Vector, dot

def step_function(x:float) -> float:
    return 1.0 if x >= 0 else 0.0

def perceptron_output(weights:Vector, bias:float,x:Vector) -> float:
    calculations = dot(weights, x) + bias
    return step_function(calculations)

In [6]:
#AND_Gate
and_weights = [2. , 2] 
and_bias = - 3.
assert perceptron_output(and_weights, and_bias, [1, 1] ) == 1 
assert perceptron_output(and_weights, and_bias, [0, 1] ) == 0 
assert perceptron_output(and_weights, and_bias, [1, 0] ) == 0
assert perceptron_output(and_weights, and_bias, [0, 0] ) == 0

In [9]:
#OR_Gate
or_weights = [2.,2]
or_bias = -1
assert perceptron_output(or_weights, or_bias, [1, 1] ) == 1 
assert perceptron_output(or_weights, or_bias, [0, 1] ) == 1
assert perceptron_output(or_weights, or_bias, [1, 0] ) == 1
assert perceptron_output(or_weights, or_bias, [0, 0] ) == 0

In [11]:
#NOT_Gate
not_weights = [-2.]
not_bias = 1.
assert perceptron_output(not_weights, not_bias, [0] ) == 1
assert perceptron_output(not_weights, not_bias, [1] ) == 0

<b>Building the Feed Forward Neural Networks</b>

In [14]:
import math

def sigmoid(t:float) -> float:  # a smooth acivation function
    return 1 / (1+math.exp(-t))

def neuron_output(weights:Vector, inputs:Vector) -> float:
    return sigmoid(dot(weights,inputs))

In [15]:
from typing import List

def feed_forward(neural_network: List[List[Vector]],
                 input_vector: Vector) -> List[Vector]:
    
    outputs: List[Vector] = []
    for layer in neural_network:
        input_with_bias = input_vector + [1]
        output = [neuron_output(neuron,input_with_bias)
                  for neuron in layer]
        outputs.append(output)
        input_vector = output
    return outputs

In [20]:
xor_network = [
    [[20.,20,-30],
    [20.,20,-10]],
    [[-60.,60,-30]]]

assert 0.000 < feed_forward(xor_network,[0,0])[-1][0] < 0.001
assert 0.999 < feed_forward(xor_network,[1,0])[-1][0] < 1.000
assert 0.999 < feed_forward(xor_network,[0,1])[-1][0] < 1.000
assert 0.000 < feed_forward(xor_network,[1,1])[-1][0] < 0.001